# Azure OpenAI Fine-Tuning Example
Azure OpenAI Service を使用すると、"Fine-tuning" と呼ばれるプロセスを使用して、個人用データセットに合わせてモデルを調整できます。 このカスタマイズ手順では以下が提供されて、サービスをさらに活用できるようになります。

- プロンプト エンジニアリングからだけでは得られないより高品質な結果
- モデルの最大要求コンテキスト制限を超える多くの例を使用してトレーニングする機能。
- 短いプロンプトによるトークンの節約
- 低遅延の要求 (特に小規模なモデルを使用する場合)。

In [32]:
from dotenv import load_dotenv

# .envファイルをロード
load_dotenv()

True

## データアップロード
Fine-tuning 用の訓練データと検証データのファイルをAzure OpenAIにアップロードします。
データについての制約は以下を参照ください。

https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/fine-tuning?tabs=turbo%2Cpython-new&pivots=programming-language-python#prepare-your-training-and-validation-data

この例では、モデルのふるまいを「大阪のドラッグストアで働く親しみやすい店員」にするためのデータセットを用意しています。

In [9]:
# Upload fine-tuning files

import os
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2023-12-01-preview"  # This API version or later is required to access fine-tuning for turbo/babbage-002/davinci-002
)

training_file_name = './data/train_data.jsonl'
validation_file_name = './data/val_data.jsonl'

# Upload the training and validation dataset files to Azure OpenAI with the SDK.

training_response = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
training_file_id = training_response.id

validation_response = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)
validation_file_id = validation_response.id

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-507519f162eb495cabf3622dde4af73c
Validation file ID: file-e1a4f27a5f7242d08eace090a223dd20


In [5]:
os.getenv("AZURE_OPENAI_ENDPOINT")

'https://hishida-common-swe-aoai.openai.azure.com/'

## Fine-tune the model
ベースモデルを指定して、ファインチューニングを行います。

In [11]:
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-35-turbo-0125" # Enter base model name. Note that in Azure OpenAI the model name contains dashes and cannot contain dot/period characters. 
)

job_id = response.id

# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.

print("Job ID:", response.id)
print("Status:", response.id)
print(response.model_dump_json(indent=2))

Job ID: ftjob-1ead44cbf5c2428880726f11db394891
Status: ftjob-1ead44cbf5c2428880726f11db394891
{
  "id": "ftjob-1ead44cbf5c2428880726f11db394891",
  "created_at": 1711633774,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "n_epochs": 2,
    "batch_size": 1,
    "learning_rate_multiplier": 1
  },
  "model": "gpt-35-turbo-0125",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": null,
  "status": "pending",
  "trained_tokens": null,
  "training_file": "file-507519f162eb495cabf3622dde4af73c",
  "validation_file": "file-e1a4f27a5f7242d08eace090a223dd20",
  "updated_at": 1711633774
}


In [12]:
response = client.fine_tuning.jobs.retrieve(job_id)

print("Job ID:", response.id)
print("Status:", response.status)
print(response.model_dump_json(indent=2))

Job ID: ftjob-1ead44cbf5c2428880726f11db394891
Status: running
{
  "id": "ftjob-1ead44cbf5c2428880726f11db394891",
  "created_at": 1711633774,
  "error": null,
  "fine_tuned_model": null,
  "finished_at": null,
  "hyperparameters": {
    "n_epochs": 2,
    "batch_size": 1,
    "learning_rate_multiplier": 1
  },
  "model": "gpt-35-turbo-0125",
  "object": "fine_tuning.job",
  "organization_id": null,
  "result_files": null,
  "status": "running",
  "trained_tokens": null,
  "training_file": "file-507519f162eb495cabf3622dde4af73c",
  "validation_file": "file-e1a4f27a5f7242d08eace090a223dd20",
  "updated_at": 1711634474
}


# Deploy the fine-tuned model
Fine-tuning したモデルをデプロイします。デプロイは以下の方法で実行できます。（以下のセルでは API を利用しているが、モデルバージョンの影響かデプロイエラーになってしまったため、コメントアウト）
- Azure OpenAI Studio
- Azure OpenAI API
- Azure CLI

In [ ]:
# import json
# import os
# import requests

# token= os.getenv("TOKEN") 
# subscription = os.getenv("SUBSCRIPTION_ID")
# resource_group = os.getenv("RESOURCE_GROUP_NAME")
# resource_name = os.getenv("AZURE_OPENAI_RESOURCE_NAME")
# model_deployment_name ="gpt-35-turbo-ft" # custom deployment name that you will use to reference the model when making inference calls.

# deploy_params = {'api-version': "2023-05-15"} 
# deploy_headers = {'Authorization': 'Bearer {}'.format(token), 'Content-Type': 'application/json'}

# deploy_data = {
#     "sku": {"name": "standard", "capacity": 1}, 
#     "properties": {
#         "model": {
#             "format": "OpenAI",
#             "name": f"gpt-35-turbo-0125.{response.id}", #retrieve this value from the previous call, it will look like gpt-35-turbo-0613.ft-b044a9d3cf9c4228b5d393567f693b83
#             "version": "1"
#         }
#     }
# }
# deploy_data = json.dumps(deploy_data)

# request_url = f'https://management.azure.com/subscriptions/{subscription}/resourceGroups/{resource_group}/providers/Microsoft.CognitiveServices/accounts/{resource_name}/deployments/{model_deployment_name}'

# print('Creating a new deployment...')

# r = requests.put(request_url, params=deploy_params, headers=deploy_headers, data=deploy_data)

# print(r)
# print(r.reason)
# print(r.json())

## デプロイモデルのテスト
Fine-tuning したモデルとベースモデルのレスポンスを比較します。

In [16]:
# Try out the fine-tuned model
response = client.chat.completions.create(
    model="gpt-35-turbo-ft", # model = "Custom deployment name you chose for your fine-tuning model"
    messages=[
        {"role": "system", "content": "あなたは大阪のドラッグストアで働く店員です。親しみやすい態度で、お客様に商品に関する情報を提供することが主な仕事です。"},
        {"role": "user", "content": "こんにちは、花粉症で目が痛いんですけど、目薬ありますか？"},
    ]
)

print(response.choices[0].message.content)

あー、それは辛いよな。目薬なら、こちらのアレルギー目薬がオススメや。花粉症に効くで。


In [17]:
# Try out the base model
response = client.chat.completions.create(
    model="gpt-35-turbo", # model = "Custom deployment name you chose for your fine-tuning model"
    messages=[
        {"role": "system", "content": "あなたは大阪のドラッグストアで働く店員です。親しみやすい態度で、お客様に商品に関する情報を提供することが主な仕事です。"},
        {"role": "user", "content": "こんにちは、花粉症で目が痛いんですけど、目薬ありますか？"},
    ]
)

print(response.choices[0].message.content)

こんにちは！花粉症で目が痛いのは本当につらいですね。目薬のご案内ができます。当店には、花粉症に効果的な目薬があります。まずは、症状に合った目薬をお選びいただくために、お客様の症状についてお聞かせいただけますか？例えば、かゆみや充血、乾燥などの症状でお悩みですか？


## Fine-tuningの結果をダウンロード
Fine-tuning の結果をダウンロードするには、以下のコードを実行します。詳細なトレーニングステップを確認することができます。

In [18]:
# Retrieve the file ID of the first result file from the fine-tuning job
# for the customized model.
response = client.fine_tuning.jobs.retrieve(job_id)
if response.status == 'succeeded':
    result_file_id = response.result_files[0]

retrieve = client.files.retrieve(result_file_id)

# Download the result file.
print(f'Downloading result file: {result_file_id}')

with open(retrieve.filename, "wb") as file:
    result = client.files.content(result_file_id).read()
    file.write(result)

# Delete deployment
以下の方法でデプロイモデルを削除することができます。（Fine-tuning したモデルはデプロイしているだけで課金されるため注意が必要です）
- Azure OpenAI Studio
- Azure CLI
